<a href="https://colab.research.google.com/github/ShehaniLEON/Capstone-Project_FINAL/blob/main/Diabetes%20prediction%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#run the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# LOADING THE DATASET

file_name = 'https://github.com/ShehaniLEON/Capstone-Project_FINAL/main/diabetes.csv'

# Load CSV File
df = pd.read_csv(file_name)
df.head()

HTTPError: ignored

In [ ]:
# size of the dataset
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
# identify any null records
df.isna().sum()

In [ ]:
# identify duplicates
duplicates = df[df.duplicated()]
duplicates

In [ ]:
# create a unique ID for each patient record
df.index

In [ ]:
df['Patient ID'] = df.index+1
df.head()

In [ ]:
print ("total number of rows :",format(len(df)))
print("number of rows where Glucose is zero: {0}",format(len(df.loc[df['Glucose'] == 0])))
print("number of rows where BloodPressure is zero: {0}",format(len(df.loc[df['BloodPressure'] == 0])))
print("number of rows where SkinThickness is zero: {0}",format(len(df.loc[df['SkinThickness'] == 0])))
print("number of rows where Insulin is zero: {0}",format(len(df.loc[df['Insulin'] == 0])))
print("number of rows where BMI is zero: {0}",format(len(df.loc[df['BMI'] == 0])))
print("number of rows where DiabetesPedigreeFunction is zero: {0}",format(len(df.loc[df['DiabetesPedigreeFunction'] == 0])))
print("number of rows where Age is zero: {0}",format(len(df.loc[df['Age'] == 0])))

In [ ]:
#we see for some patients, certain attributes has the value zero
#we will be replacing the zero's with the median value.

df["Insulin"]= df["Insulin"].replace(0,df["Insulin"].median())
df["Glucose"]= df["Glucose"].replace(0,df["Glucose"].median())
df["BloodPressure"]= df["BloodPressure"].replace(0,df["BloodPressure"].median())
df["SkinThickness"]= df["SkinThickness"].replace(0,df["SkinThickness"].median())
df

Data Visualization

In [ ]:
f,ax0 = plt.subplots()
ax1 = ax0.twinx()

df['Glucose'].plot(kind='hist',ax=ax0)
df['Glucose'].plot(kind='kde', c='red', ax=ax1)

In [ ]:
f,ax0 = plt.subplots()
ax1 = ax0.twinx()

df['BMI'].plot(kind='hist', bins=12, ax=ax0)
df['BMI'].plot(kind='kde', c='red', ax=ax1)

In [ ]:
df['BloodPressure'].plot(kind='hist')

In [ ]:
df['SkinThickness'].plot(kind='hist')

In [ ]:
df['Insulin'].plot(kind='hist')

In [ ]:
fig= plt.figure(figsize=(16,6))

sns.distplot(df["Age"][df["Outcome"]==1])
plt.title("Age")

In [ ]:
df.plot(x='SkinThickness', y='Insulin', kind='scatter')

In [ ]:
df.plot(x='BloodPressure', y='Insulin', kind='scatter')

Treating outliers(removing outliers)

In [ ]:
def remove_outlier (dataFrame):
  for column_name in dataFrame.columns:
    Q1= df[column_name].quantile(0.25)
    Q3= df[column_name].quantile(0.75)
    IQR=Q3-Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR
    print(f"{column_name}>>lower limit:{lower_limit} \n upper limit:{upper_limit}")
    dataFrame=dataFrame[(dataFrame[column_name] > lower_limit)|(dataFrame[column_name] <upper_limit)]

    return dataFrame

In [ ]:
df

In [ ]:
df.columns

In [ ]:
correlation_matrix = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']].corr()
correlation_matrix
df.corr()

In [ ]:
correlation_matrix["Outcome"].sort_values(ascending=False)

In [ ]:
sns.heatmap(correlation_matrix)

In [ ]:
X= df.drop(['Outcome','Insulin','BloodPressure'], axis=1)
X

In [ ]:
Y=df['Outcome']
Y

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8, random_state=42)

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
print(F"Train sample size = {len(X_train)}")
print(F"Test sample size  = {len(X_test)}")

Model building

(1) Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model =LogisticRegression()

In [ ]:
#train the model using training data
model.fit(X_train,Y_train)

In [ ]:
# predict using testing data
predictions= model.predict(X_test)
predictions

Evaluating the model

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(Y_test,predictions)
cm

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
y_pred_prob_0 = model.predict_proba(X_test)[:, 0]
y_pred_prob_1 = model.predict_proba(X_test)[:, 1]


test_result = pd.DataFrame(data={'y_act':Y_test, 'y_pred':predictions, 'y_pred_prob_0':y_pred_prob_0 , 'y_pred_prob_1':y_pred_prob_1})
test_result.sample(10)

In [ ]:
cfm = pd.crosstab(test_result['y_act'], test_result['y_pred'], margins=True)
cfm

In [ ]:
accuracy = (cfm.loc[0,0] + cfm.loc[1,1])/ cfm.loc['All','All'] 
accuracy

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(Y_test,predictions))

(2) Random forest classifier

In [ ]:
#standard scaling
from sklearn.preprocessing import StandardScaler

sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train,Y_train)

In [ ]:
rf_y_pred =rf_classifier.predict(X_test)

In [ ]:
rf_cm=confusion_matrix(rf_y_pred,Y_test)
rf_cm

In [ ]:
sns.heatmap(rf_cm, annot=True)

In [ ]:
print("Accuracy:", sum(rf_y_pred== Y_test)/len(rf_y_pred))